In [1]:
import cv2 #opencv
from keras.models import load_model
import numpy as np #for complex matrix calculation
from collections import deque

Using TensorFlow backend.


In [3]:
model1=load_model('devanagari.h5') #loading the model
print(model1) #printing the model

In [4]:
letter_count={0:'CHECK', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                    7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                    11: '11_taamatar',
                    12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                    18: '18_da',
                    19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                    23: '23_ba',
                    24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                    31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                    34: '34_chhya', 35: '35_tra', 36: '36_gya', 37:'CHECK'} #defining a dictionary

In [5]:
def keras_presict(model, image): #function to predict the data from the webcam
    processed=keras_process_image(image)
    print("processed:" + str(processed.shape))
    pred_probab=model.predict(processed)[0]
    pred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class #returning to the calling variable

def keras_process_image(img): #function for processing the image
    image_x=32 #width of the image
    image_y=32 #height of the image
    img=cv2.resize(img, (image_x, image_y))
    img=np.array(img, dtype=np.float32)
    img=np.reshape(img, (-1, image_x, image_y, 1))
    return img #rerturning to the calling variable

In [6]:
cap=cv2.VideoCapture(0) #opening the webcam
Lower_blue=np.array([110, 50, 50]) #for the detection of the blue colour
Upper_blue=np.array([130, 255, 255])
pred_class=0
pts=deque(maxlen=512)
blackboard=np.zeros((480, 640, 3), dtype=np.uint8)
digit=np.zeros((200, 200, 3), dtype=np.uint8)
while (cap.isOpened()): 
    ret, img=cap.read() #reading the image from webcam
    img=cv2.flip(img, 1) #flipping the image
    imgHSV=cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #converting to HSV format
    mask=cv2.inRange(imgHSV, Lower_blue, Upper_blue)
    blur=cv2.medianBlur(mask, 15) #blurring the background
    blur=cv2.GaussianBlur(blur, (5, 5), 0)
    thresh=cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts=cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0] #counting contours
    print(cnts)
    center=None
    if len(cnts) >= 1: #if contours are present
        contour = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(contour)>250:
            ((x, y), radius)=cv2.minEnclosingCircle(contour)
            cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2) #creating a circle around the contours
            cv2.circle(img, center, 5, (0, 0, 255), -1)
            M=cv2.moments(contour)
            center=(int(M['m10']/M['m00']), int(M['m01']/M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i-1], pts[i], (255, 255, 255), 10)
                cv2.line(img, pts[i-1], pts[i], (0, 0, 255), 5)
    elif len(cnts)==0: #if contours are not detected or blue colour is not present
        if len(pts)!=[]:
            blackboard_gray=cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1=cv2.medianBlur(blackboard_gray, 15)
            blur1=cv2.GaussianBlur(blur1, (5, 5), 0)
            thresh1=cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts=cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts)>=1:
                cnt=max(blackboard_cnts, key=cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt)>2000: #removing the blue colour in the background
                    x, y, w, h=cv2.boundingRect(cnt)
                    digit=blackboard_gray[y:y +h, x:x +w]
                    pred_probab, pred_class=keras_predict(model1, digit)
                    print(pred_class, pred_probab)
        pts=deque(maxlen=512)
        blackboard=np.zeros((480, 640, 3), dtype=np.uint8)
    cv2.putText(img, "Conv Network:" + str(letter_count[pred_class]), (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k=cv2.waitKey(10)
    if k==27:
        break
            

[array([[[267,  93]],

       [[267,  94]],

       [[267,  95]],

       [[267,  96]],

       [[268,  96]],

       [[269,  96]],

       [[270,  96]],

       [[271,  96]],

       [[271,  95]],

       [[271,  94]],

       [[271,  93]],

       [[270,  93]],

       [[269,  93]],

       [[268,  93]]], dtype=int32)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[array([[[484,   0]],

       [[483,   1]],

       [[483,   2]],

       [[482,   3]],

       [[481,   3]],

       [[480,   4]],

       [[479,   5]],

       [[480,   6]],

       [[480,   7]],

       [[481,   7]],

       [[482,   7]],

       [[483,   7]],

       [[484,   7]],

       [[485,   6]],

       [[486,   6]],

       [[487,   5]],

       [[488,   4]],

       [[488,   3]],

       [[488,   2]],

       [[489,   1]],

       [[489,   0]],

       [[488,   0]],

       [[487,   0]],

       [[486,   0]],

       [[485,   0]]], dtype=int32)]
[array([[[484,   0]],

       [[483,   1]],

       [[483,   2]],



[array([[[108,  52]],

       [[107,  53]],

       [[108,  54]],

       [[109,  54]],

       [[110,  53]],

       [[109,  52]]], dtype=int32), array([[[476,  30]],

       [[475,  31]],

       [[476,  32]],

       [[477,  32]],

       [[478,  32]],

       [[479,  32]],

       [[480,  31]],

       [[479,  30]],

       [[478,  30]],

       [[477,  30]]], dtype=int32)]
[array([[[108,  52]],

       [[107,  53]],

       [[108,  54]],

       [[109,  54]],

       [[110,  53]],

       [[109,  52]]], dtype=int32), array([[[476,  30]],

       [[475,  31]],

       [[476,  32]],

       [[477,  32]],

       [[478,  32]],

       [[479,  32]],

       [[480,  31]],

       [[479,  30]],

       [[478,  30]],

       [[477,  30]]], dtype=int32)]
[]
[]
[array([[[109,  30]],

       [[108,  31]],

       [[108,  32]],

       [[108,  33]],

       [[109,  34]],

       [[108,  35]],

       [[107,  35]],

       [[106,  35]],

       [[105,  35]],

       [[104,  36]],

       [[10

[array([[[460,  54]],

       [[460,  55]],

       [[461,  55]],

       [[461,  54]]], dtype=int32), array([[[142,  26]],

       [[141,  27]],

       [[140,  28]],

       [[139,  29]],

       [[138,  30]],

       [[137,  31]],

       [[138,  32]],

       [[139,  33]],

       [[140,  34]],

       [[141,  35]],

       [[142,  35]],

       [[143,  35]],

       [[144,  35]],

       [[145,  35]],

       [[146,  35]],

       [[147,  35]],

       [[148,  34]],

       [[149,  33]],

       [[149,  32]],

       [[150,  31]],

       [[150,  30]],

       [[149,  29]],

       [[149,  28]],

       [[148,  27]],

       [[147,  27]],

       [[146,  26]],

       [[145,  26]],

       [[144,  26]],

       [[143,  26]]], dtype=int32), array([[[461,   0]],

       [[462,   1]],

       [[463,   2]],

       [[464,   2]],

       [[465,   3]],

       [[466,   3]],

       [[467,   3]],

       [[468,   3]],

       [[469,   3]],

       [[470,   3]],

       [[471,   3]],

   

[array([[[472, 184]],

       [[472, 185]],

       [[472, 186]],

       [[473, 187]],

       [[474, 187]],

       [[475, 186]],

       [[475, 185]],

       [[475, 184]],

       [[474, 184]],

       [[473, 184]]], dtype=int32), array([[[124,  30]],

       [[123,  31]],

       [[122,  31]],

       [[121,  32]],

       [[120,  33]],

       [[120,  34]],

       [[119,  35]],

       [[118,  35]],

       [[117,  36]],

       [[117,  37]],

       [[118,  38]],

       [[118,  39]],

       [[118,  40]],

       [[119,  41]],

       [[119,  42]],

       [[120,  42]],

       [[121,  43]],

       [[122,  43]],

       [[123,  44]],

       [[124,  44]],

       [[125,  45]],

       [[126,  45]],

       [[127,  45]],

       [[128,  45]],

       [[129,  45]],

       [[130,  44]],

       [[130,  43]],

       [[131,  42]],

       [[130,  41]],

       [[130,  40]],

       [[131,  39]],

       [[131,  38]],

       [[131,  37]],

       [[131,  36]],

       [[132,  35

       [[513,   0]]], dtype=int32)]
[array([[[115,  17]],

       [[115,  18]],

       [[116,  19]],

       [[117,  19]],

       [[118,  20]],

       [[119,  20]],

       [[120,  20]],

       [[121,  19]],

       [[121,  18]],

       [[121,  17]],

       [[120,  17]],

       [[119,  17]],

       [[118,  17]],

       [[117,  17]],

       [[116,  17]]], dtype=int32), array([[[512,   0]],

       [[512,   1]],

       [[513,   1]],

       [[514,   2]],

       [[515,   2]],

       [[516,   2]],

       [[517,   2]],

       [[518,   2]],

       [[519,   2]],

       [[520,   1]],

       [[521,   1]],

       [[522,   1]],

       [[523,   1]],

       [[524,   1]],

       [[525,   1]],

       [[525,   0]],

       [[524,   0]],

       [[523,   0]],

       [[522,   0]],

       [[521,   0]],

       [[520,   0]],

       [[519,   0]],

       [[518,   0]],

       [[517,   0]],

       [[516,   0]],

       [[515,   0]],

       [[514,   0]],

       [[513,   0]]], dty

...

[array([[[564, 411]],

       [[563, 412]],

       [[562, 413]],

       [[561, 413]],

       [[560, 414]],

       [[559, 414]],

       [[558, 414]],

       [[557, 415]],

       [[556, 415]],

       [[555, 415]],

       [[555, 416]],

       [[554, 417]],

       [[555, 418]],

       [[554, 419]],

       [[554, 420]],

       [[554, 421]],

       [[555, 422]],

       [[555, 423]],

       [[556, 423]],

       [[557, 424]],

       [[558, 424]],

       [[559, 424]],

       [[560, 424]],

       [[561, 424]],

       [[562, 424]],

       [[563, 424]],

       [[564, 424]],

       [[565, 423]],

       [[566, 423]],

       [[567, 423]],

       [[568, 422]],

       [[569, 421]],

       [[569, 420]],

       [[570, 419]],

       [[570, 418]],

       [[571, 417]],

       [[571, 416]],

       [[571, 415]],

       [[571, 414]],

       [[571, 413]],

       [[570, 413]],

       [[569, 412]],

       [[568, 411]],

       [[567, 411]],

       [[566, 411]],

       [[

[array([[[586, 477]],

       [[585, 478]],

       [[584, 479]],

       [[585, 479]],

       [[586, 479]],

       [[587, 479]],

       [[588, 479]],

       [[588, 478]],

       [[587, 478]]], dtype=int32), array([[[558, 474]],

       [[557, 475]],

       [[556, 475]],

       [[555, 476]],

       [[554, 477]],

       [[553, 478]],

       [[553, 479]],

       [[554, 479]],

       [[555, 479]],

       [[556, 479]],

       [[557, 479]],

       [[558, 479]],

       [[559, 479]],

       [[560, 479]],

       [[561, 479]],

       [[562, 479]],

       [[563, 479]],

       [[564, 479]],

       [[564, 478]],

       [[563, 477]],

       [[563, 476]],

       [[562, 475]],

       [[561, 474]],

       [[560, 474]],

       [[559, 474]]], dtype=int32), array([[[ 86,   0]],

       [[ 86,   1]],

       [[ 87,   2]],

       [[ 88,   2]],

       [[ 89,   3]],

       [[ 90,   3]],

       [[ 91,   3]],

       [[ 92,   3]],

       [[ 93,   3]],

       [[ 94,   3]],

   

[array([[[83, 19]],

       [[82, 20]],

       [[81, 20]],

       [[81, 21]],

       [[81, 22]],

       [[81, 23]],

       [[81, 24]],

       [[81, 25]],

       [[82, 25]],

       [[83, 25]],

       [[84, 25]],

       [[84, 24]],

       [[84, 23]],

       [[84, 22]],

       [[85, 21]],

       [[85, 20]],

       [[84, 19]]], dtype=int32), array([[[ 92,   0]],

       [[ 92,   1]],

       [[ 92,   2]],

       [[ 91,   3]],

       [[ 90,   4]],

       [[ 89,   4]],

       [[ 88,   5]],

       [[ 88,   6]],

       [[ 88,   7]],

       [[ 89,   7]],

       [[ 90,   8]],

       [[ 91,   8]],

       [[ 92,   8]],

       [[ 93,   8]],

       [[ 94,   8]],

       [[ 95,   7]],

       [[ 96,   7]],

       [[ 97,   7]],

       [[ 98,   7]],

       [[ 99,   7]],

       [[100,   7]],

       [[101,   7]],

       [[102,   7]],

       [[103,   7]],

       [[104,   7]],

       [[105,   7]],

       [[106,   7]],

       [[107,   6]],

       [[108,   6]],

       

       [[52,  0]]], dtype=int32)]
[array([[[471, 373]],

       [[470, 374]],

       [[469, 375]],

       [[469, 376]],

       [[468, 377]],

       [[468, 378]],

       [[468, 379]],

       [[468, 380]],

       [[468, 381]],

       [[469, 382]],

       [[470, 383]],

       [[471, 384]],

       [[471, 385]],

       [[472, 386]],

       [[472, 387]],

       [[473, 388]],

       [[474, 388]],

       [[475, 388]],

       [[476, 387]],

       [[477, 387]],

       [[478, 386]],

       [[478, 385]],

       [[479, 384]],

       [[479, 383]],

       [[479, 382]],

       [[480, 381]],

       [[480, 380]],

       [[480, 379]],

       [[479, 378]],

       [[478, 377]],

       [[478, 376]],

       [[477, 375]],

       [[477, 374]],

       [[476, 374]],

       [[475, 373]],

       [[474, 373]],

       [[473, 373]],

       [[472, 373]]], dtype=int32), array([[[51,  0]],

       [[52,  1]],

       [[53,  1]],

       [[54,  2]],

       [[55,  2]],

       [[56,  1

NameError: name 'keras_predict' is not defined